In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fuzzywuzzy import fuzz, process
from IPython.display import display,HTML
import spacy
from spacy.lang.en import English
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import pickle

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
pd.options.display.max_rows = 300
PATH = 'data/'

In [4]:
data = [pd.read_csv(f'{PATH}PDdataset_{n}.csv',low_memory=False) for n in range(1,4)]
train, train_label, test = data
nlp = spacy.load('xx')
tokenizer = English().Defaults.create_tokenizer()

# Looking at the dataset

In [5]:
for n in data:
    display(n.head())

,platform,app_id,app_name,publisher_name,categories
0,gp,com.cgames.bonza,Crossword Jigsaw,Best Word Game Studio,"GAME, GAME_WORD"
1,gp,com.anoshenko.android.solitaires,250+ Solitaire Collection,Alexei Anoshenko,"GAME_CARD, GAME"
2,gp,com.xunxin.doudizu,Landlords,SwiftStar,"GAME_CARD, GAME"
3,gp,com.netk.kumamon_speed_second,くまモンのスピード（無料トランプゲーム）,Kazutaka Sato,"GAME_CARD, GAME"
4,gp,br.com.tapps.epicpartyclicker,Epic Party Clicker - Throw Epic Dance Parties!,Tapps Games,"GAME_MUSIC, GAME"


,gp_app_id,ios_app_id,is_match
0,com.linternaute.dictionnaire,352732499,True
1,cl.ibex.servipag,873133446,True
2,com.swyft.nfl,1044513914,True
3,com.peekaboo.cookapp,833499556,True
4,com.games2win.mirandasingsvshaters,1044202386,True


,platform,app_id,app_name,publisher_name,categories
0,gp,com.talking.wowacat,WOWaCAT,SIA Reklāmas Rīma,"GAME_MUSIC, GAME"
1,gp,com.arcade.xiaoxiaobuyu,多人捕魚_小小遊戲聯網版-深海機台小霸王掌上1000打魚達人,Judy Deng,"GAME, GAME_CASINO"
2,gp,com.wsohbetmobil1_4535913,sohbetmobil1.com sohbetmobil1,Umut Turk,"APPLICATION, DATING"
3,gp,com.turner.mixelrush,Mixels Rush,Cartoon Network,"GAME_ACTION, GAME, FAMILY, FAMILY_ACTION"
4,gp,air.com.tutotoons.app.princesscleanuptime,Princess Castle Cleanup,TutoTOONS Kids Games,"GAME_EDUCATIONAL, GAME, FAMILY, FAMILY_ACTION"


In [6]:
for n in data:
    display(DataFrameSummary(n).summary())

,platform,app_id,app_name,publisher_name,categories
count,500,500,500,500,500
unique,2,500,482,430,195
top,ios,740974884,Emoji Shuffle!,G5 Entertainment,"APPLICATION, BUSINESS"
freq,252,1,2,8,17
counts,500,500,500,500,500
uniques,2,500,482,430,195
missing,0,0,0,0,0
missing_perc,0%,0%,0%,0%,0%
types,bool,unique,categorical,categorical,categorical


,gp_app_id,ios_app_id,is_match
count,NaN,101,NaN
mean,NaN,8.34658e+08,NaN
std,NaN,2.403e+08,NaN
min,NaN,3.32546e+08,NaN
25%,NaN,6.42313e+08,NaN
50%,NaN,9.04214e+08,NaN
75%,NaN,1.04231e+09,NaN
max,NaN,1.17902e+09,NaN
counts,101,101,101
uniques,101,101,1


,platform,app_id,app_name,publisher_name,categories
count,500,500,500,500,500
unique,2,500,486,444,185
top,gp,com.argurth.queferiezvoussi,beanfun! App,Ketchapp,"APPLICATION, EDUCATION"
freq,252,1,2,6,21
counts,500,500,500,500,500
uniques,2,500,486,444,185
missing,0,0,0,0,0
missing_perc,0%,0%,0%,0%,0%
types,bool,unique,categorical,categorical,categorical


# Feature engineering

### Defining overlapping categories

In [7]:
ios_cat = {
          "Books":1,
          "Business":2,
          "Catalogs":1,
          "Education":4,
          "Entertainment":5,
          "Finance":6,
          "Food & Drink":7,
          "Games":8,
          "Health & Fitness":9,
          "Lifestyle":10,
          "Medical":11,
          "Music":12,
          "Navigation":13,
          "News":14,
          "Photo & Video":15,
          "Productivity":16,
          "Reference":1,
          "Social Networking":18,
          "Sports":19,
          "Travel":20,
          "Utilities":21,
          "Weather":22,
          "Action":23,
          "Adventure":24,
          "Arcade":25,
          "Board":26,
          "Card":27,
          "Casino":28,
          "Dice":26,
          "Educational":30,
          "Family":31,
          "Kids":31,
          "Puzzle":33,
          "Racing":34,
          "Role Playing":35,
          "Simulation":36,
          "Strategy":37,
          "Trivia":38,
          "Word":39
}

In [8]:
gp_cat = {
    "GAME":8,
    "ART_AND_DESIGN":5,
    "AUTO_AND_VEHICLES":13,
    "BEAUTY":10,
    "BOOKS_AND_REFERENCE":1,
    "BUSINESS":2,
    "COMICS":1,
    "COMMUNICATION":18,
    "DATING":18,
    "EDUCATION":4,
    "ENTERTAINMENT":5,
    "EVENTS":5,
    "FINANCE":6,
    "FOOD_AND_DRINK":7,
    "HEALTH_AND_FITNESS":9,
    "HOUSE_AND_HOME":10,
    "LIFESTYLE":10,
    "MAPS_AND_NAVIGATION":13,
    "MEDICAL":11,
    "MUSIC_AND_AUDIO":12,
    "NEWS_AND_MAGAZINES":14,
    "PARENTING":10,
    "PERSONALIZATION":10,
    "PHOTOGRAPHY":15,
    "PRODUCTIVITY":16,
    "SHOPPING":10,
    "SOCIAL":18,
    "SPORTS":19,
    "TOOLS":21,
    "TRAVEL_AND_LOCAL":20,
    "VIDEO_PLAYERS":15,
    "WEATHER":22,
    "LIBRARIES_AND_DEMO":1,
    "GAME_ARCADE":25,
    "GAME_PUZZLE":33,
    "GAME_CARD":27,
    "GAME_CASUAL":38,
    "GAME_RACING":34,
    "GAME_SPORTS":38,
    "GAME_ACTION":23,
    "GAME_ADVENTURE":24,
    "GAME_BOARD":26,
    "GAME_CASINO":28,
    "GAME_EDUCATIONAL":30,
    "GAME_MUSIC":38,
    "GAME_ROLE_PLAYING":35,
    "GAME_SIMULATION":36,
    "GAME_STRATEGY":37,
    "GAME_TRIVIA":38,
    "GAME_WORD":39,
    "FAMILY":31,
    "FAMILY_UNDER_5":31,
    "FAMILY_6_TO_8":31,
    "FAMILY_9_AND_UP":31,
    "FAMILY_ACTION":31,
    "FAMILY_BRAINGAMES":31,
    "FAMILY_CREATE":31,
    "FAMILY_EDUCATION":31,
    "FAMILY_MUSICVIDEO":31,
    "FAMILY_PRETEND":31
}

In [9]:
def jaccard(x,y):
    """ Jaccard-index for overlapping numeric categories. """
    intersection= len(set.intersection(x,y))
    union= len(set.union(x,y))
    return intersection/union

### Using spacy and fuzzywuzzy for similarity features of app names & publisher names

In [10]:
"""uncommand for making train-set
gp = train.loc[train['platform']=='gp'] 
ios = train.loc[train['platform']=='ios']
"""
gp = test.loc[test['platform']=='gp'] 
ios = test.loc[test['platform']=='ios']

In [11]:
# renaming the columns before concatenating gp/ios pairs
gp.drop(['platform'],axis=1,inplace=True)
gp.rename(columns=lambda x: x+'_gp',inplace=True)
ios.drop(['platform'],axis=1,inplace=True)
ios.rename(columns=lambda x: x+'_ios',inplace=True)

df_ratio = pd.DataFrame()

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [8]:
#going trough all possible matches
for g in range(len(gp)):
    for i in range(len(ios)):
        pair = pd.concat([gp.iloc[g:g+1].reset_index(drop=True),
                          ios.iloc[i:i+1].reset_index(drop=True)], axis=1)
        
        #translate into connected categories
        cat_gp_numeric = set([gp_cat[k] for k in re.split(", |/",pair.at[0,"categories_gp"]) 
                              if(k in gp_cat)])
        cat_ios_numeric = set([ios_cat[k] for k in re.split(", |/",pair.at[0,"categories_ios"]) 
                               if(k in ios_cat)])
        pair.at[0,'cat_gp'] = str(cat_gp_numeric)
        pair.at[0,'cat_ios'] = str(cat_ios_numeric)
        
        #calculate the similarity as new features
        pair['app_name_ratio'] = fuzz.token_set_ratio(pair.at[0,'app_name_gp'],
                                                      pair.at[0,'app_name_ios'])
        pair['app_name_spacy'] = nlp(pair.at[0,'app_name_gp']).similarity(nlp(pair.at[0,'app_name_ios']))
        pair['publisher_name_ratio'] = fuzz.token_set_ratio(pair.at[0,'publisher_name_gp'],
                                                            pair.at[0,'publisher_name_ios'])
        pair['publisher_name_spacy'] = nlp(pair.at[0,'publisher_name_gp']).similarity(nlp(pair.at[0,'publisher_name_ios']))
        pair['categories_ratio'] = fuzz.token_set_ratio(pair.at[0,'categories_gp'],
                                                        pair.at[0,'categories_ios'])
        pair['categories_spacy'] = nlp(pair.at[0,'categories_gp']).similarity(nlp(pair.at[0,'categories_ios']))
        pair["cat_jaccard"] = jaccard(cat_gp_numeric,cat_ios_numeric)
        
        df_ratio = df_ratio.append(pair)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [ ]:
df_ratio = df_ratio.reindex(sorted(df_ratio.columns),axis=1)
df_ratio.reset_index(drop=True,inplace=True)

In [12]:
"""uncommand for making train-set
df_ratio.to_feather(f'{PATH}ratio')
"""

df_ratio.to_feather(f'{PATH}ratio_test')

# Random forest

In [12]:
df_ratio = pd.read_feather(f'{PATH}ratio')

In [13]:
#Looking at data sorted by new features
df_ratio.sort_values(by=['app_name_ratio',
                         'app_name_spacy',
                         'publisher_name_ratio',
                         'publisher_name_spacy',
                         'cat_jaccard',
                         'categories_ratio',
                         'categories_spacy'],ascending=False).reset_index(drop=True).head()

,app_id_gp,app_id_ios,app_name_gp,app_name_ios,app_name_ratio,app_name_spacy,cat_gp,cat_ios,cat_jaccard,categories_gp,categories_ios,categories_ratio,categories_spacy,publisher_name_gp,publisher_name_ios,publisher_name_ratio,publisher_name_spacy
0,com.pomelogames.bulletboy.google,899606169,Bullet Boy,Bullet Boy,100,1.0,"{8, 25}","{8, 25, 34}",0.666667,"GAME_ARCADE, GAME","Games/Racing, Games/Arcade, Games",51,0.447328,Kongregate,Kongregate,100,1.0
1,com.noodlecake.brickies,969122371,Brickies,Brickies,100,1.0,"{8, 25}","{8, 25, 23}",0.666667,"GAME_ARCADE, GAME","Games/Action, Games/Arcade, Games",46,0.429491,Noodlecake Studios Inc,Noodlecake Studios Inc,100,1.0
2,com.vancouver.knows,1052391422,温哥华知道 vancoolver,温哥华知道 vancoolver,100,1.0,{10},"{10, 14}",0.500000,"APPLICATION, LIFESTYLE","News, Lifestyle",78,0.302112,ZhiDao Media Inc.,ZhiDao Media Inc.,100,1.0
3,com.swyft.nfl,1044513914,NFL Emojis,NFL Emojis,100,1.0,{19},"{19, 5}",0.500000,"APPLICATION, SPORTS","Entertainment, Sports",53,0.438400,NFL Enterprises LLC,NFL Enterprises LLC,100,1.0
4,com.gameloft.android.ANMP.GloftOOHM,915713314,Battle Odyssey,Battle Odyssey,100,1.0,"{8, 35}","{8, 33, 35, 5}",0.500000,"GAME, GAME_ROLE_PLAYING","Entertainment, Games/Puzzle, Games/Role Playin...",49,0.512254,Gameloft,Gameloft,100,1.0


In [14]:
#preparing Dataset
df_ratio.app_id_ios = df_ratio.app_id_ios.astype(int)
df_ratio_label = pd.merge(df_ratio,
                          train_label.rename(columns={'gp_app_id':'app_id_gp',
                                                      'ios_app_id':'app_id_ios'}),
                          on =['app_id_gp','app_id_ios'],how='left')
df_ratio_label.is_match = df_ratio_label.is_match.fillna(False)
df_ratio_label.is_match = df_ratio_label.is_match.astype(int)
df_random_forest = df_ratio_label.drop(['app_id_gp','app_id_ios',
                                        'app_name_gp','app_name_ios',
                                        'cat_gp','cat_ios',
                                        'categories_gp','categories_ios',
                                        'publisher_name_gp','publisher_name_ios'],axis=1)


In [15]:
model = RandomForestClassifier(n_estimators=40,oob_score=True)

### Make Validationset

In [16]:
def split_random(df,set_size):
    indices = sorted(np.random.permutation(len(df))[:set_size])
    return df.iloc[indices].copy()

def get_train_val(df,fraction=0.2):
    """ Splitting dataset into trainingsset& validationset. """
    val = split_random(df,int(len(df)*fraction))
    train = df.drop(list(val.index))
    val.reset_index(drop=True,inplace=True)
    train.reset_index(drop=True,inplace=True)
    return train,val

In [17]:
train,val = get_train_val(df_random_forest)

### Train the model

In [21]:
%time model.fit(train.drop(['is_match'],axis=1),train['is_match'])

CPU times: user 1.08 s, sys: 0 ns, total: 1.08 s
Wall time: 1.07 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

### Roc_auc_score for trainingsset and validationset

In [19]:
metrics.roc_auc_score(train['is_match'],
                      model.predict_proba(train.drop(['is_match'],axis=1))[:,1]),
metrics.roc_auc_score(val['is_match'],
                      model.predict_proba(val.drop(['is_match'],axis=1))[:,1])

0.9999746963562753

In [20]:
model.oob_score_

0.9994599675980559

### Feature importance

In [21]:
pd.DataFrame({'name':val.drop(['is_match'],axis=1).columns,
              'importance':model.feature_importances_}
            ).sort_values(by=['importance'],ascending=False).reset_index(drop=True)

,name,importance
0,app_name_ratio,0.367597
1,publisher_name_ratio,0.243848
2,app_name_spacy,0.173404
3,publisher_name_spacy,0.104056
4,categories_spacy,0.045941
5,cat_jaccard,0.032905
6,categories_ratio,0.032249


### Displaying results on trainset

In [22]:
df_ratio_label['predict'] = model.predict(df_random_forest.drop(['is_match'],axis=1))
df_ratio_label['predict_proba'] = model.predict_proba(df_random_forest.drop(['is_match'],axis=1))[:,1]

In [23]:
df_ratio_label.sort_values(by=['is_match',
                               'predict',
                               'predict_proba'],ascending=False).head()

,app_id_gp,app_id_ios,app_name_gp,app_name_ios,app_name_ratio,app_name_spacy,cat_gp,cat_ios,cat_jaccard,categories_gp,categories_ios,categories_ratio,categories_spacy,publisher_name_gp,publisher_name_ios,publisher_name_ratio,publisher_name_spacy,is_match,predict,predict_proba
1,com.noodlecake.brickies,969122371,Brickies,Brickies,100,1.0,"{8, 25}","{8, 25, 23}",0.666667,"GAME_ARCADE, GAME","Games/Action, Games/Arcade, Games",46,0.429491,Noodlecake Studios Inc,Noodlecake Studios Inc,100,1.0,1,1,1.0
3,com.swyft.nfl,1044513914,NFL Emojis,NFL Emojis,100,1.0,{19},"{19, 5}",0.500000,"APPLICATION, SPORTS","Entertainment, Sports",53,0.438400,NFL Enterprises LLC,NFL Enterprises LLC,100,1.0,1,1,1.0
4,com.gameloft.android.ANMP.GloftOOHM,915713314,Battle Odyssey,Battle Odyssey,100,1.0,"{8, 35}","{8, 33, 35, 5}",0.500000,"GAME, GAME_ROLE_PLAYING","Entertainment, Games/Puzzle, Games/Role Playin...",49,0.512254,Gameloft,Gameloft,100,1.0,1,1,1.0
5,tw.txwy.and.hsqj,1086003937,橫掃千軍,橫掃千軍,100,1.0,"{8, 35}","{8, 37, 35, 5}",0.500000,"GAME, GAME_ROLE_PLAYING","Entertainment, Games/Role Playing, Games/Strat...",48,0.543697,Longcheng Ltd.,Longcheng Ltd.,100,1.0,1,1,1.0
6,com.g5e.pilotbr2.full,640543364,Pilot Brothers 2 (Full),Pilot Brothers 2 (Full),100,1.0,"{8, 24}","{8, 33, 5, 24}",0.500000,"GAME, GAME_ADVENTURE","Entertainment, Games/Puzzle, Games/Adventure, ...",36,0.429770,G5 Entertainment,G5 Entertainment,100,1.0,1,1,1.0


In [30]:
pickle.dump(model,open(f'{PATH}model_rf','wb'))

# Testset prediction

In [24]:
df_ratio = pd.read_feather(f'{PATH}ratio_test')

In [18]:
model = pickle.load(open(f'{PATH}model_rf','rb'))

In [25]:
#Looking at data sorted by new features
df_ratio.sort_values(by=['app_name_ratio',
                         'app_name_spacy',
                         'publisher_name_ratio',
                         'publisher_name_spacy',
                         'cat_jaccard',
                         'categories_ratio',
                         'categories_spacy'],ascending=False).reset_index(drop=True).head()

,app_id_gp,app_id_ios,app_name_gp,app_name_ios,app_name_ratio,app_name_spacy,cat_gp,cat_ios,cat_jaccard,categories_gp,categories_ios,categories_ratio,categories_spacy,publisher_name_gp,publisher_name_ios,publisher_name_ratio,publisher_name_spacy
0,com.scottgames.ragequitv2,841164653,Rage Quit,Rage Quit,100,1.0,"{8, 25}","{8, 25, 23}",0.666667,"GAME_ARCADE, GAME","Games/Action, Games/Arcade, Games",46,0.429491,Scott Cawthon,Scott Cawthon,100,1.0
1,com.inthepocket.kinepolis,368204284,Kinepolis,Kinepolis,100,1.0,{5},"{10, 5}",0.500000,"APPLICATION, ENTERTAINMENT","Entertainment, Lifestyle",72,0.345285,Kinepolis Group N.V.,Kinepolis Group N.V.,100,1.0
2,com.marlon0008.armyguide,1086400508,Army Guide,Army Guide,100,1.0,{1},"{1, 4}",0.500000,"APPLICATION, BOOKS_AND_REFERENCE","Education, Reference",64,0.352883,Marlon Clark,Marlon Clark,100,1.0
3,com.g5e.brightstonemysteries.android,674535826,Brightstone Mysteries: Paranormal Hotel,Brightstone Mysteries: Paranormal Hotel,100,1.0,"{8, 24}","{8, 33, 5, 24}",0.500000,"GAME, GAME_ADVENTURE","Entertainment, Games/Puzzle, Games/Adventure, ...",36,0.429770,G5 Entertainment,G5 Entertainment,100,1.0
4,com.ketchapp.rushhero,998729635,Rush Hero,Rush Hero,100,1.0,"{8, 25}","{8, 25, 5, 23}",0.500000,"GAME_ARCADE, GAME","Entertainment, Games/Action, Games/Arcade, Games",33,0.427978,Ketchapp,Ketchapp,100,1.0


In [26]:
df_random_forest = df_ratio.drop(['app_id_gp',
                                  'app_id_ios',
                                  'app_name_gp',
                                  'app_name_ios',
                                  'cat_gp',
                                  'cat_ios',
                                  'categories_gp',
                                  'categories_ios',
                                  'publisher_name_gp',
                                  'publisher_name_ios'],axis=1)
df_ratio['predict'] = model.predict(df_random_forest)
df_ratio['predict_proba'] = model.predict_proba(df_random_forest)[:,1]

In [27]:
df_ratio.sort_values(by=['predict_proba'],ascending=False).head()

,app_id_gp,app_id_ios,app_name_gp,app_name_ios,app_name_ratio,app_name_spacy,cat_gp,cat_ios,cat_jaccard,categories_gp,categories_ios,categories_ratio,categories_spacy,publisher_name_gp,publisher_name_ios,publisher_name_ratio,publisher_name_spacy,predict,predict_proba
8161,com.ketchapp.rushhero,998729635,Rush Hero,Rush Hero,100,1.000000,"{8, 25}","{8, 25, 5, 23}",0.5,"GAME_ARCADE, GAME","Entertainment, Games/Action, Games/Arcade, Games",33,0.427978,Ketchapp,Ketchapp,100,1.0,1,1.0
51733,com.gismart.guitar.tuner,662890455,Chromatic Guitar Tuner Free: Ukulele Bass Violin,Chromatic Guitar Tuner: Ukulele Bass Violin,100,0.936332,{12},"{12, 21}",0.5,"APPLICATION, MUSIC_AND_AUDIO","Utilities, Music",33,0.425365,Gismart,Gismart,100,1.0,1,1.0
4499,com.realarcade.CLB,518119428,Collapse! Blast: Match 3 Free,Collapse! Blast,100,0.756115,"{8, 33}","{8, 33, 18, 25}",0.5,"GAME, GAME_PUZZLE","Games/Puzzle, Social Networking, Games/Arcade,...",49,0.352489,GameHouse,GameHouse,100,1.0,1,1.0
24294,com.g5e.brightstonemysteries.android,674535826,Brightstone Mysteries: Paranormal Hotel,Brightstone Mysteries: Paranormal Hotel,100,1.000000,"{8, 24}","{8, 33, 5, 24}",0.5,"GAME, GAME_ADVENTURE","Entertainment, Games/Puzzle, Games/Adventure, ...",36,0.429770,G5 Entertainment,G5 Entertainment,100,1.0,1,1.0
61676,com.marlon0008.armyguide,1086400508,Army Guide,Army Guide,100,1.000000,{1},"{1, 4}",0.5,"APPLICATION, BOOKS_AND_REFERENCE","Education, Reference",64,0.352883,Marlon Clark,Marlon Clark,100,1.0,1,1.0


In [28]:
df_test_label = df_ratio[['app_id_gp',
                          'app_id_ios',
                          'predict']][df_ratio['predict']==1].reset_index(drop=True)
df_test_label.rename(columns={'app_id_gp':'gp_app_id',
                              'app_id_ios':'ios_app_id',
                              'predict':'is_match'},inplace=True)
df_test_label.is_match=df_test_label.is_match.astype(bool)
df_test_label.head()

,gp_app_id,ios_app_id,is_match
0,com.talking.wowacat,638418451,True
1,com.turner.mixelrush,919132359,True
2,com.quizup.core,718421443,True
3,com.mattel.bestjobever,927523832,True
4,air.com.spilgames.PuppetSoccerChampions2015,904241944,True


In [52]:
df_test_label.to_csv(f'{PATH}test_label.csv',index=False)